In [1]:
#####ベイジアン変量効果二項回帰モデル#####
##みどり本10章:個体差と生存種子数
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
from numpy.random import *
from scipy import optimize
from scipy.stats import norm


In [2]:
####データの発生####
#np.random.seed(8742)   #シードを設定

#データの設定
N = 500   #観測個体数
n = poisson(10, N)   #個体ごとの種子数

In [3]:
##応答変数の生成
#パラメータの設定
alpha = alphat = normal(0, 0.4, N)   #変量効果
beta = betat = -0.5   #全体共通の切片

#ロジットモデルと確率を設定
logit = alpha + beta   #ロジット
Pr = np.exp(logit) / (1 + np.exp(logit))   #確率

#二項分布から応答変数を生成
y = binomial(n, Pr, N)
pd.DataFrame({'1. 種子数' : n, '2. 生存数' : y, '3. 観測確率' : y/n, '4. 真の確率' : Pr})   #データの確認

,1. 種子数,2. 生存数,3. 観測確率,4. 真の確率
0,12,3,0.250000,0.319787
1,8,3,0.375000,0.243961
2,15,7,0.466667,0.349809
3,8,5,0.625000,0.331750
4,13,5,0.384615,0.402289
5,6,2,0.333333,0.400267
6,17,2,0.117647,0.269577
7,6,3,0.500000,0.321770
8,7,4,0.571429,0.273485
9,11,0,0.000000,0.393066


In [4]:
####マルコフ連鎖モンテカルロ法で変量効果二項回帰モデルを推定####
##二項回帰モデルの対数尤度関数を設定
def define_likelihood(alpha, beta, n, y):
    #ロジットと確率の定義
    logit = alpha + beta
    Pr = np.exp(logit)/(1+np.exp(logit))
    
    #対数尤度の計算
    LLi = y * np.log(Pr) + (n-y) * np.log(1-Pr)
    LL = sum(LLi)   
    return LL, LLi

In [5]:
##アルゴリズムの設定
R = 10000
keep = 4  
iter = 0
burnin = 2000/keep
disp = 100

##事前分布の設定
tau1 = 100   #パラメータの事前分布
s0 = 0.01   #階層モデルの事前分布
v0 = 0.01
rw1 = 0.1   #ランダムウォーク幅
rw2 = 0.15

##初期値の設定
alpha = normal(0, 0.2, N)
beta = 0
tau2 = 0.5   #階層モデルの分散の初期値

##パラメータの保存用配列
ALPHA = np.zeros((int(R/keep), N))
BETA = np.zeros((int(R/keep)))
LL = np.zeros((int(R/keep)))

In [6]:
####メトロポリスヘイスティング法でパラメータをサンプリング####
for rp in range(R):
    ##全体共通のbetaをサンプリング
    #betaをサンプリング
    betad = beta
    betan = betad + normal(0, rw1)

    #対数尤度と対数事前分布の計算
    lognew1 = define_likelihood(alpha, betan, n, y)
    logold1 = define_likelihood(alpha, betad, n, y)
    logpnew1 = -0.5 * (np.power(betan, 2) / 2*tau1)
    logpold1 = -0.5 * (np.power(betad, 2) / 2*tau1)

    #MHサンプリングでbetaを採択するかどうかを決定
    gamma = min(1, np.exp(lognew1[0] + logpnew1 - logold1[0] - logpold1))
    u = uniform(0, 1, 1)

    #alpha > u なら新しいbetaを採択
    if gamma > u:
        beta = betan
        logl = lognew1
    else :
        beta = betad
        logl = logold1


    ##変量効果alphaをサンプリング
    #alphaをサンプリング
    alphad = alpha
    alphan = alphad + normal(0, rw2, N)

    #対数尤度と対数事前分布の計算
    lognew2 = define_likelihood(alphan, beta, n, y)
    logold2 = logl
    logpnew2 = -0.5 * (np.power(alphan, 2) / 2*tau2)
    logpold2 = -0.5 * (np.power(alphad, 2) / 2*tau2)

    #MHサンプリングでbetaを採択するかどうかを決定
    rand = uniform(0, 1, N)   #一様分布から乱数を発生
    LLind_diff = np.exp(lognew2[1] + logpnew2 - logold2[1] - logpold2)   #採択率
    gamma = (LLind_diff > 1)*1 + (LLind_diff <= 1)*LLind_diff

    #gammaの値に基づき新しいalphaを採択
    flag = (gamma >= rand)*1 + (gamma < rand)*0
    alpha = flag*alphan + (1-flag)*alphad


    ##逆ガンマ分布から階層モデルの分散をサンプリング
    s = s0 + sum(np.power(alpha - np.mean(alpha), 2))   
    v = v0 + N 
    tau2 = 1/np.random.gamma(v/2, 1/(s/2), 1)   #逆ガンマ分布からtauをサンプリング
    
    ##サンプリング結果の保存
    if rp%keep==0:
        mkeep = int(rp/keep)
        BETA[mkeep] = beta
        ALPHA[mkeep, :] = alpha
        LL[mkeep] = logl[0]
    
    if rp%disp==0:
        print(rp)
        print(np.array([beta, tau2, ]))
        print(logl[0])

0
[ 0.         0.0553953]
-3421.3904625
100
[-0.43693524  0.83239766]
-3125.38907346
200
[-0.45826198  0.93650037]
-3132.66979417
300
[-0.43385794  0.92806628]
-3111.04674736
400
[-0.42238328  0.96293397]
-3112.19233605
500
[-0.36904221  1.04820421]
-3134.8250547
600
[-0.30465905  0.92285246]
-3135.49314444
700
[-0.35800797  0.86483063]
-3136.6754174
800
[-0.37875029  1.00232618]
-3149.00915074
900
[-0.48156399  1.01039155]
-3123.22928258
1000
[-0.51446509  1.08564067]
-3144.90970203
1100
[-0.45163958  0.89702571]
-3129.92673084
1200
[-0.41294981  1.01184873]
-3120.17919513
1300
[-0.4050426   1.09089493]
-3144.28932345
1400
[-0.39472468  0.91045178]
-3137.4628296
1500
[-0.38287908  0.89773289]
-3152.50585847
1600
[-0.35398093  0.92141031]
-3116.63570036
1700
[-0.46803343  0.9919299 ]
-3130.6614744
1800
[-0.55226535  1.09650074]
-3124.09997069
1900
[-0.49894322  0.94256541]
-3142.07177696
2000
[-0.52533181  1.01644324]
-3143.84998371
2100
[-0.40252638  0.93606079]
-3151.54017983
2200
[-

In [16]:
x = np.array([[1, 2, 3], [4, 5, 6]])
random.multinomial(1, x[1, :], 1)


AttributeError: 'builtin_function_or_method' object has no attribute 'multinomial'

In [25]:
np.sum(x, axis=1)

array([ 6, 15])

In [23]:

x

array([[1, 2, 3],
       [4, 5, 6]])

267.5978272999356

In [207]:
s

74.505704270656921